# Haskell koan: probabilstic programming in < 100 LoC

Let us begin with our incantations to GHC:

In [18]:
{-# LANGUAGE GADTs #-}
import Control.Monad (ap)

First, we create a new monad called `P`, for *probability*.
It supports only two operations:
- `Ret` to convert a pure value into a `P` value,
- `Sample01` to sample from a distribution.

We encode the monad in CPS style to get a monad instance for-free. For more
modularity, use the free monad technique.

In [19]:
data P a where
  Ret :: a -> P a -- ^ life a pure value
  Sample01 :: (Float -> P a) -> P a -- ^ use a float uniformly sampled from [0, 1]
  
instance Functor P where
  fmap f (Ret x) = Ret (f x)
  fmap f (Sample01 rand2pa) = Sample01 $ \r -> fmap f (rand2pa r)
  
instance Monad P where
  return = Ret
  Ret a >>= a2pb = a2pb a
  Sample01 r2pa >>= a2pb = Sample01 $ \r -> r2pa r >>= a2pb

instance Applicative P where
  pure = return
  (<*>) = ap